We will use a Kaggle dataset about credit card fraud detection. This dataset consists of transactions made by credit cards in September 2013 by European cardholders.
It contains 492 frauds out of a total of 284,807 transactions. You can see that it is highly unbalanced, where the positive class (frauds) account for 0.172% of all transactions. https://www.kaggle.com/datasets/whenamancodes/fraud-detection
The objective of this article is illustrate how to train a built-in model like XGBoost in an AWS Sagemaker's notebook instance. In this case a supervised learning, specifically a binary classification problem.

But first, what do I mean by built-in algorithms? Amazon SageMaker provides several built-in algorithms that you can use to train your models. These algorithms are highly optimized, scalable, and designed to generate accurate models rapidly, eliminating the need to create and maintain the underlying algorithm containers, thus saving time and resources.

Let's start by importing the basic python libraries.

In [2]:
import pandas as pd
import numpy as np

We use pandas to read the dataset:

In [99]:
df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


We should do an Exploratory Data Analysis here, howerver we will skip it for brevity. We will drop the feature "Time" and will retain the rest of anonymised credit card related features.

In [100]:
df = df.drop(columns=['Time'])
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Let's check the number of transactions belonging to each class (Fraud or Normal):

In [101]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

It is heavily imbalanced. Since this article aims to focus on the Sagemaker side, we won't try to optimise the model to get the best possible result. Therefore, let's just undersample randomly the majority class. We will take randomly the same number of transactions that we have in the normal ones, i.e. 492.

In [103]:
# Fraud transactions
df_1 = df[df.Class == 1]

# Undersampled normal transactions
df_0 = df[df.Class == 0].sample(df_1.shape[0])

Now we concatenate both datasets and reset the index:

In [105]:
df_bal = pd.concat((df_0, df_1), axis=0)
df_bal.reset_index(drop=True, inplace=True)
df_bal.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.876591,-0.155384,0.361307,0.987282,-0.045266,0.446142,0.007623,0.193986,-0.276750,0.004941,...,0.109980,0.132488,-0.060499,-0.278848,0.303804,-0.436598,0.034929,0.031098,120.00,0
1,-0.670310,0.803298,2.078889,0.397425,0.464632,-0.511305,1.025419,-0.186385,-0.195366,-0.615139,...,-0.028493,0.232540,-0.238697,0.606070,0.209983,-0.525307,-0.058154,-0.116236,1.00,0
2,1.955515,-0.577371,-0.083715,0.242985,-0.553677,0.599422,-1.221767,0.295218,2.577602,-0.207888,...,-0.103016,-0.019276,0.312469,0.158574,-0.622843,0.425357,-0.052634,-0.053608,14.95,0
3,-3.189870,1.321957,0.471306,2.477934,0.081151,0.793808,-0.133616,0.991140,-1.147900,1.202786,...,0.100378,0.301430,-0.268094,-0.268996,0.310630,0.188044,-1.377141,-0.330257,1.00,0
4,-0.444927,0.364529,-0.698316,0.780500,-0.263478,0.549751,1.026421,0.288750,-1.271627,-0.061335,...,0.549802,1.131155,0.402189,0.366272,-0.544862,1.108977,-0.038090,0.136139,255.33,0


Before continuing, since we are going to use XGBoost, which is a decision tree based algorithm, we don't necessarily need to scale the features.

#### Split into training, validation and testing sets

First, we need to split into features (X) and target (y).

In [107]:
X = df_bal.iloc[:,:-1]
y = df_bal.iloc[:,-1]

Now, using sklearn function train_test_split, we can split our data into the training and testing sets. 

In [108]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.10, 
                                                    random_state=42, 
                                                    stratify=y)

We use 10% of the data as a testing set. Also, note that we use the argument `stratify`. This is to ensure that each set contains approximately the same percentage of samples of each target class as the complete set (y).

Since we want a validation set too, we will further split the training set. Again allocating 10%, this time as validation.

In [37]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.1, 
                                                  random_state=42, 
                                                  stratify=y_train) 

We can have a look at the size of each set to verify that we have carried that over correctly.

In [110]:
X_train.shape[0], X_test.shape[0], X_val.shape[0]

(885, 99, 89)

Amazon S3 bucket, part of AWS's Simple Storage Service (S3), is a scalable cloud storage solution providing object storage, allowing users to store and retrieve data in units termed as objects, instead of traditional file systems. Each object within the S3 bucket is identified with a unique, user-assigned key, ensuring precise data retrieval.

We will create an S3 bucket where we will store our model files. The prefix refers to the subdirectories or subfolders where the files will be.

In [111]:
bucket = 'sagemaker-test-david'
prefix = 'test'

Sagemaker built-in algorithms require datasets where the first feature is the label or target (y) and the rest of them are the independent variables (X). For the training and validation sets to be used by the algorithms, we need to upload them to an S3 bucket in a special way.

First, let's create a Sagemaker session in this notebook:

In [112]:
import sagemaker
sess = sagemaker.Session()

We need to concatenate the y and X to create the dataset in the expected format and order and save it locally as a csv file. Afterwards we need to upload it to a specific S3 bucket:

In [113]:
# Training set
pd.concat([y_train, X_train], axis=1).to_csv('train.csv', 
                                             index=False, 
                                             header=False)
sess.upload_data(path='train.csv', 
                 bucket=bucket, 
                 key_prefix=prefix+'/train')

# Validation set
pd.concat([y_val, X_val], axis=1).to_csv('validation.csv', 
                                         index=False, 
                                         header=False)
sess.upload_data(path='validation.csv', 
                 bucket=bucket, 
                 key_prefix=prefix+'/validation')

's3://sagemaker-test-david/test/validation/validation.csv'

Next, we use he Sagemaker's `TrainingInput` class to configure a data input flow for training and validation:

In [114]:
from sagemaker.inputs import TrainingInput

# Training set
s3_input_train = TrainingInput(s3_data=f's3://{bucket}/{prefix}/train', 
                               content_type='csv')

# Validation set
s3_input_validation = TrainingInput(s3_data=f's3://{bucket}/{prefix}/validation/', 
                                    content_type='csv')

#### Training

Time to train our model. In this stage we need to import our built-in algorithm. 

Before that, we will need the following information:
* region: the AWS Region where the SageMaker notebook instance is running (eu-west-2, eu-central-1, us-east-1...).
* session: the SageMaker session that holds the configuration and is used to create all the needed AWS resources like training jobs, endpoints, etc.
* role: the IAM Role ARN used to give training and deployment access to your data. You usually get this role when you create a notebook instance unless specified otherwise.
* instance type: is the type of EC2 instance used for training your model, such as "ml.m4.xlarge", "ml.m5.large", etc. You'll need to carefully choose the optimal for your case, checking parameters like memory, CPU and, of course, price per hour. https://aws.amazon.com/sagemaker/pricing/

In [124]:
import boto3

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
instance_type = 'ml.m4.large'

Next, we need to retrieve the container image URI for the specified built-in algorithm or model, in this case XGBoost. Each built-in algorithm or pre-built model in SageMaker is stored in a Docker container, and this container is located at a specific URI (Uniform Resource Identifier).

When you are creating an estimator, you need to specify the location (URI) of the Docker container that has the algorithm or model. The image_uris.retrieve function simplifies this process by generating the correct URI for you based on the algorithm or model, region, and other parameters you specify.

In [125]:
container = sagemaker.image_uris.retrieve(framework="xgboost", 
                                          region=region, 
                                          version="latest")

An Estimator is an abstraction represented by the Estimator class in the SageMaker Python SDK, which allows users to train machine learning models. It encapsulates the training job and its associated configurations, such as the algorithm container, training data, hyperparameters, and compute resources.

In [126]:
from sagemaker.estimator import Estimator

xgb = Estimator(
    container,
    role,
    sagemaker_session=sess,
    instance_count=1,
    instance_type=instance_type,
    input_mode='File',
    output_path=f's3://{bucket}/{prefix}/output',
    train_use_spot_instance=True
)

Let's explain some of the arguments we set:
* instance_count: The number of Amazon EC2 instances you want to use for training. 1 indicates a single instance.
* input_mode: The input mode that the algorithm supports. "File" mode means the training data is transferred to the training instances using Amazon S3.
* output_path: The S3 location for saving the training results (model artifacts and output files).
* train_use_spot_instance: This argument suggests using EC2 Spot Instances for training, which can significantly reduce the cost of training models.

We need to set up also the hyperparameters. These are some of the hyperparameters you can set up:
* alpha: The L1 regularization term on weights, used to avoid overfitting.
* eta: The learning rate, controlling the contribution of each tree in the ensemble.
* gamma: The minimum loss reduction required to make a further partition on a leaf node of the tree.
* max_depth: The maximum depth of a tree, controlling overfitting.
* min_child_weight: The minimum sum of instance weight needed in a child, used to control overfitting.
* subsample: The fraction of training data to randomly sample in each round to prevent overfitting.
* objective: The learning task and the corresponding learning objective. Here, we chose binary classification with logistic regression.
* num_round: The number of boosting rounds or trees to build, equivalent to the number of models in the ensemble.

In [137]:
xgb.set_hyperparameters(
    alpha=1.5,
    eta=0.1,
    gamma=4,
    max_depth=2,
    min_child_weight=3,
    subsample=0.8,
    objective='binary:logistic',
    num_round=100,
)


Finally, time to fit our data:

In [138]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2023-09-22 15:28:25 Starting - Starting the training job...
2023-09-22 15:28:51 Starting - Preparing the instances for trainingProfilerReport-1695396505: InProgress
......
2023-09-22 15:29:51 Downloading - Downloading input data...
2023-09-22 15:30:16 Training - Downloading the training image...
2023-09-22 15:30:56 Training - Training image download completed. Training in progress...Arguments: train
[2023-09-22:15:31:01:INFO] Running standalone xgboost training.
[2023-09-22:15:31:01:INFO] File size need to be processed in the node: 0.5mb. Available memory size in the node: 254.8mb
[2023-09-22:15:31:01:INFO] Determined delimiter of CSV input is ','
[15:31:01] S3DistributionType set as FullyReplicated
[15:31:01] 885x29 matrix with 25665 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-09-22:15:31:01:INFO] Determined delimiter of CSV input is ','
[15:31:01] S3DistributionType set as FullyReplicated
[15:31:01] 89x29 matrix with 2581 entries loaded fr

#### Deployment

Time to deploy the trained model so it can be used in production. We need to select the instance type and count depending our requirements. Also the model and endpoint names:

In [139]:
model_name = 'prs-test-3'

In [140]:
# create endpoint and predictor
xgb_predictor = xgb.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    model_name=model_name,
    endpoint_name=model_name,
)

-----!

#### Predictions

Before getting the predictions, we need to specify the serialization format that the SageMaker Predictor will use when sending data to our model endpoint. In this case, we'll be using the CSVSerializer, meaning that the predictor will serialize the input data to CSV format before sending it to the model endpoint for inference. Serialization is the process of converting a data structure or object state into a format that can be easily stored or transmitted, and subsequently reconstructed. In other words, it's about translating complex data types such as objects or structures into a format (like a string) that can be easily rendered into a file, sent over the network, or saved into a database. Once the serialized data reaches its destination, it can be deserialized back into its original format.

In [155]:
from sagemaker.predictor import CSVSerializer 
xgb_predictor.serializer = CSVSerializer()

Time to get the predictions for our testing set. We use the method `predict` of our deployed predictor. However, this method requires the data to be in a numpy array format, not in a pandas Dataframe. Finally, we need to decode the output and convert it to numpy array.

In [156]:
predictions = xgb_predictor.predict(X_test.values)
y_pred = np.fromstring(predictions.decode('utf-8'), sep=',')

Now we can assess the performance of the model. Since our test data was balanced at the beginning, we will calculate the accuracy for simplicity.

In [161]:
accuracy = (y_pred.round() == y_test).sum() / y_test.shape[0] * 100
print(f'The accuracy of the model is {accuracy:.2f}%')

The accuracy of the model is 94.95%


We can also see the confusion matrix:

In [170]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_pred.round())
print(conf_matrix)

[[49  1]
 [ 4 45]]


This was a toy dataset and it was possibly highly preprocessed. Therefore the result is very good.

#### Create predictor from endpoint

Imagine you created your model and endpoint in the past and you want to test some data in a notebook. You could load your endpoint in a predictor and do the same as we've just done when assessing the performance.

In [171]:
from sagemaker.predictor import Predictor

# Load endpoint
predictor = Predictor(
    endpoint_name=model_name,
)

# Set serialization to CSV
predictor.serializer = CSVSerializer()

Get the predictions as done before:

In [172]:
predictions = predictor.predict(X_test.values)
y_pred = np.fromstring(predictions.decode('utf-8'), sep=',')

The next step would be to make the endpoint available for a production environment. You can achieve this using an AWS Lambda Function and creating an API from it. We will cover this in future articles.